In [97]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import numpy as np

In [98]:
#If using Google CoLab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Update the below file path with the path to the .csv file of SDSS data. Change object name for galaxy or quasar training.

In [99]:
all_data = pd.read_csv('path/to/Star_ugriz.csv', skiprows=1)

In [100]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   objid       500000 non-null  int64  
 1   modelmag_u  500000 non-null  float64
 2   modelmag_g  500000 non-null  float64
 3   modelmag_r  500000 non-null  float64
 4   modelmag_i  500000 non-null  float64
 5   modelmag_z  500000 non-null  float64
 6   z           500000 non-null  float64
 7   weight      500000 non-null  float64
dtypes: float64(7), int64(1)
memory usage: 30.5 MB


In [101]:
all_data.head()

,objid,modelmag_u,modelmag_g,modelmag_r,modelmag_i,modelmag_z,z,weight
0,1237650795682726190,19.36548,18.94647,19.04925,19.21883,19.35806,0.000111,0.000075
1,1237648722289295504,20.36442,18.21354,17.38881,17.11562,17.32000,0.000178,0.000009
2,1237648721752424633,19.31302,18.13527,17.67289,17.49996,17.44187,0.000213,0.000013
3,1237648721752424612,24.85999,20.76403,19.18368,17.93489,17.28289,0.000066,0.000024
4,1237651752921595910,18.32052,17.01957,16.47373,16.26664,16.14915,0.000286,0.000010


In [102]:
all_data.describe()


,objid,modelmag_u,modelmag_g,modelmag_r,modelmag_i,modelmag_z,z,weight
count,5.000000e+05,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000
mean,1.237663e+18,19.796510,18.293621,17.718352,17.193991,17.016120,-0.000081,0.019168
std,7.163714e+12,82.650601,77.620962,69.422828,78.888036,77.604367,0.000432,1.524789
min,1.237646e+18,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-0.004329,-6.000000
25%,1.237658e+18,18.871380,17.481167,16.877667,16.575897,16.392785,-0.000229,0.000009
50%,1.237663e+18,20.191685,18.778190,18.137045,17.822950,17.592800,-0.000053,0.000014
75%,1.237667e+18,21.920365,20.229610,19.474220,18.979550,18.719683,0.000086,0.000029
max,1.237681e+18,32.118950,32.576640,31.981080,31.468180,30.846780,0.004621,867.654700


Determine appropriate cutoff quantity for zErr (originally labelled as "weight" because it was going to be used in the training process)

In [80]:
all_data['weight'].quantile(.95)

0.00010321813999999998

Drop sentinel values and potential catastrophic failures

In [103]:
no_sent = all_data.where(all_data > -1)
#Comment the line below out if using the unfiltered version of the dataset
#no_sent['weight'] = no_sent['weight'].where(no_sent['weight']<0.00011)
no_sent = no_sent.dropna()

Define new calculated columns that represent "color" in the field of astronomy by subtracting adjacent photometric bands.

In [104]:
no_sent['u-g'] = no_sent['modelmag_u']-no_sent['modelmag_g']
no_sent['g-r'] = no_sent['modelmag_g']-no_sent['modelmag_r']
no_sent['r-i'] = no_sent['modelmag_r']-no_sent['modelmag_i']
no_sent['i-z'] = no_sent['modelmag_i']-no_sent['modelmag_z']
no_sent.head()


,objid,modelmag_u,modelmag_g,modelmag_r,modelmag_i,modelmag_z,z,weight,u-g,g-r,r-i,i-z
0,1237650795682726190,19.36548,18.94647,19.04925,19.21883,19.35806,0.000111,0.000075,0.41901,-0.10278,-0.16958,-0.13923
1,1237648722289295504,20.36442,18.21354,17.38881,17.11562,17.32000,0.000178,0.000009,2.15088,0.82473,0.27319,-0.20438
2,1237648721752424633,19.31302,18.13527,17.67289,17.49996,17.44187,0.000213,0.000013,1.17775,0.46238,0.17293,0.05809
3,1237648721752424612,24.85999,20.76403,19.18368,17.93489,17.28289,0.000066,0.000024,4.09596,1.58035,1.24879,0.65200
4,1237651752921595910,18.32052,17.01957,16.47373,16.26664,16.14915,0.000286,0.000010,1.30095,0.54584,0.20709,0.11749


In [105]:
# if using the classifer:
# clas = []
# for val in no_sent['z'].tolist():
#     if val > 0:
#         clas.append(1)
#     else:
#         clas.append(0)
# no_sent['class'] = clas

Create structure for ANN

In [106]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # number of hidden nodes in each layer
        # kept small since input and output layers are also small in quantity
        hidden_1 = 20
        hidden_2 = 30
        hidden_3 = 20
        # linear layer (4 -> hidden_1)
        self.fc1 = nn.Linear(4, hidden_1)
        # establish ReLU as activation function for this layer (and all)
        self.act1 = nn.ReLU()
        # linear layer (hidden_1 -> hidden_2)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.act2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.1)
        # linear layer (hidden_2 -> hidden_3)
        self.fc3 = nn.Linear(hidden_2, hidden_3)
        self.act3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.1)
        # output layer (hidden_3 -> 1)
        self.output = nn.Linear(hidden_3, 1)
         # uncomment the sigmoid to convert to a classifier
        #self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # add hidden layers, with relu activation function in same line
        x = self.act1(self.fc1(x))
        x = self.act2(self.fc2(x))
        # add dropout layer
        x = self.dropout2(x)
        #repeat for 3rd layer
        x = self.act3(self.fc3(x))
        x = self.dropout3(x)
        # add output
        x = self.output(x)
        # uncomment the sigmoid to convert to a classifier
        #x = self.sigmoid(x)
        return x

# initialize the NN
model = Net()
print(model)

Net(
  (fc1): Linear(in_features=4, out_features=20, bias=True)
  (act1): ReLU()
  (fc2): Linear(in_features=20, out_features=30, bias=True)
  (act2): ReLU()
  (dropout2): Dropout(p=0.1, inplace=False)
  (fc3): Linear(in_features=30, out_features=20, bias=True)
  (act3): ReLU()
  (dropout3): Dropout(p=0.1, inplace=False)
  (output): Linear(in_features=20, out_features=1, bias=True)
)


In [107]:
# specify loss function (mean squared error)
criterion = nn.MSELoss()

# use categorical cross-entropy if classifying
#criterion = nn.BCELoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [108]:
inputs = no_sent[no_sent.columns[-4:]]
targets = no_sent['z']
# for classifier:
#targets = no_sent['class']

In [109]:

inputs.head()
# May have to adjust no_sent column indices above for classification case

,u-g,g-r,r-i,i-z
0,0.41901,-0.10278,-0.16958,-0.13923
1,2.15088,0.82473,0.27319,-0.20438
2,1.17775,0.46238,0.17293,0.05809
3,4.09596,1.58035,1.24879,0.65200
4,1.30095,0.54584,0.20709,0.11749


In [110]:
from sklearn.model_selection import train_test_split

# create train/test split, and then split again for validation set
X_train, X_test, y_train, y_test = train_test_split(inputs, targets, train_size = 0.85, shuffle = True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size = 0.85, shuffle = True)

In [111]:
no_sent['u-g'].values[0].dtype
no_sent['z'].values[0].dtype

dtype('float64')

In [112]:
# Have to convert to float32 because the initial datatype is a double
# Turn the df columns into tensors
inputs_train = torch.tensor(X_train.values.astype(np.float32))
targets_train = torch.tensor(y_train.values.astype(np.float32))
inputs_val = torch.tensor(X_val.values.astype(np.float32))
targets_val = torch.tensor(y_val.values.astype(np.float32))

In [113]:
inputs_test = torch.tensor(X_test.values.astype(np.float32))
targets_test = torch.tensor(y_test.values.astype(np.float32))

In [114]:
# Use built-in pytorch DataLoader and TensorDataset to prepare custom data
batch_size = 32
from torch.utils.data import DataLoader, TensorDataset

train_ds = TensorDataset(inputs_train, targets_train)
val_ds = TensorDataset(inputs_val, targets_val)
train_dl = DataLoader(train_ds, batch_size, shuffle = True)
val_dl = DataLoader(val_ds, batch_size, shuffle = True)

In [115]:
test_ds = TensorDataset(inputs_test, targets_test)
test_dl = DataLoader(test_ds, batch_size, shuffle = True)

In [116]:
# number of epochs to train the model
n_epochs = 200
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    ### track the changes in the validation loss
    valid_loss = 0.0

    ###################
    # train the model #
    ###################
    model.train() # prep model for training
    for data, target in train_dl:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = (criterion(output, target.unsqueeze(1)))
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()

    ######################
    ### validate the model ### Important: we are not performing backpropagation step here; that's only for training
    ######################
    model.eval() # prep model for evaluation
    for data, target in val_dl:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = (criterion(output, target.unsqueeze(1)))
        # update running validation loss
        valid_loss += loss.item()

    ### print training/validation statistics
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_dl)
    valid_loss = valid_loss/len(val_dl)

    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1,
        train_loss,
        valid_loss
        ))

    ### save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 0.000346 	Validation Loss: 0.000031
Validation loss decreased (inf --> 0.000031).  Saving model ...
Epoch: 2 	Training Loss: 0.000058 	Validation Loss: 0.000014
Validation loss decreased (0.000031 --> 0.000014).  Saving model ...
Epoch: 3 	Training Loss: 0.000030 	Validation Loss: 0.000008
Validation loss decreased (0.000014 --> 0.000008).  Saving model ...
Epoch: 4 	Training Loss: 0.000021 	Validation Loss: 0.000006
Validation loss decreased (0.000008 --> 0.000006).  Saving model ...
Epoch: 5 	Training Loss: 0.000015 	Validation Loss: 0.000004
Validation loss decreased (0.000006 --> 0.000004).  Saving model ...
Epoch: 6 	Training Loss: 0.000013 	Validation Loss: 0.000003
Validation loss decreased (0.000004 --> 0.000003).  Saving model ...
Epoch: 7 	Training Loss: 0.000011 	Validation Loss: 0.000003
Validation loss decreased (0.000003 --> 0.000003).  Saving model ...
Epoch: 8 	Training Loss: 0.000010 	Validation Loss: 0.000002
Validation loss decreased (0.00000

In [117]:
# evaluate regression model
from sklearn.metrics import r2_score

# if calculating this not at the same time as training
#model.load_state_dict(torch.load('/content/star_final.pt'))
model.eval()
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
test_acc = 0.0



for data, target in test_dl:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output, target.unsqueeze(1))
    r2 = r2_score(target.detach().numpy(), output.detach().numpy())
    # update test loss
    test_loss += loss.item()*data.size(0)
    test_acc += r2*data.size(0)

# calculate and print avg test loss
test_loss = test_loss/len(test_dl.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))
# calculate and print avg test accuracy
test_acc = test_acc/len(test_dl.dataset)
print('Test Acc: {:.6f}\n'.format(test_acc))

Test Loss: 0.000409

Test Acc: -0.210741



In [59]:
# evaluate classification model
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))

model.eval() # prep model for evaluation

for data, target in test_dl:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    #safeguard, because last batch is a remainder and does not = 32
    if len(output) != batch_size or len(target) != batch_size:
      continue
    #print(output.data)
    # calculate the loss
    loss = criterion(output, target.unsqueeze(1))
    # update test loss
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    if (len(correct) != batch_size):
      continue
    # calculate test accuracy for each object class
    for i in range(batch_size):
        #label = int(target.data[i])
        label = int(target.detach().numpy()[0])
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate and print avg test loss
test_loss = test_loss/len(test_dl.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(2):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.661319

Test Accuracy of     0: 61% (28162/45440)
Test Accuracy of     1: 58% (15321/26144)

Test Accuracy (Overall): 60% (43483/71584)
